##Loading Data

In [19]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pangru","key":"b2b5b699d7ff5be3629527ed811fd3b2"}'}

In [20]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [21]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 49.6MB/s]


In [22]:
! mkdir tweetsDisaster

In [23]:
! unzip nlp-getting-started.zip -d tweetsDisaster

Archive:  nlp-getting-started.zip
  inflating: tweetsDisaster/sample_submission.csv  
  inflating: tweetsDisaster/test.csv  
  inflating: tweetsDisaster/train.csv  


In [24]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, utils

In [25]:
train = pd.read_csv('tweetsDisaster/train.csv')
test = pd.read_csv('tweetsDisaster/test.csv')


##DataPreprocess

In [26]:
train[train["target"]==0]["text"].values[1]
train[train["target"]==1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [27]:
#shuffle data
train=utils.shuffle(train, random_state=0)

In [28]:
#building vectors (tokens)
count_vectorizer = feature_extraction.text.CountVectorizer()
example_train_vectors=count_vectorizer.fit_transform(train["text"][:5]) #先以前五個texts測試使用

print(example_train_vectors.todense().shape) #五個texts 字典長度就有77
print(example_train_vectors[0].todense()) #第一個text有12個字(對應index上顯示1，其餘為0)
print(example_train_vectors[0])

(5, 77)
[[0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 1 1 0]]
  (0, 32)	1
  (0, 75)	1
  (0, 4)	1
  (0, 34)	1
  (0, 27)	1
  (0, 65)	1
  (0, 59)	1
  (0, 24)	1
  (0, 74)	1
  (0, 58)	1
  (0, 47)	1
  (0, 6)	1


In [29]:
train_vectors = count_vectorizer.fit_transform(train["text"])
test_vectors = count_vectorizer.transform(test["text"])

## Model

In [30]:
from sklearn.ensemble import VotingClassifier
from pandas.core.common import random_state
from sklearn.linear_model import LogisticRegression,SGDClassifier, RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier,RandomForestClassifier

# ridge_clf = linear_model.RidgeClassifier(alpha=6.5)  #
# sgd_clf=linear_model.SGDClassifier(alpha=0.1)
# # log_clf=linear_model.LogisticRegression(C=1.5)
# lr = linear_model.LogisticRegression(max_iter=1500,C=0.25,penalty = 'l2',class_weight='balanced',solver = 'liblinear',intercept_scaling = 2)
# voting_clf=VotingClassifier(estimators=[('lr',lr),('sgd',sgd_clf),('ridge',ridge_clf)],voting='hard')


lr = LogisticRegression(C=0.125,multi_class='multinomial',random_state=1)
svm = SVC(probability=True,C=1.5)
rnd_clf = RandomForestClassifier(n_estimators=600,random_state=1)
voting_clf=VotingClassifier(estimators=[('lr',lr),('rnd',rnd_clf),('svm',svm)],voting='soft')

In [31]:
# scores = model_selection.cross_val_score(ridge_clf, train_vectors, train["target"], cv=3, scoring="f1")
# print(scores)
# scores = model_selection.cross_val_score(sgd_clf, train_vectors, train["target"], cv=3, scoring="f1")
# print(scores)
# score=model_selection.cross_val_score(lr,train_vectors,train["target"],cv=3,scoring="f1")
# print(score)
# score=model_selection.cross_val_score(voting_clf,train_vectors,train["target"],cv=3,scoring="f1")
# print(score)

scores = model_selection.cross_val_score(lr, train_vectors, train["target"], cv=3, scoring="f1")
print(scores)
scores = model_selection.cross_val_score(svm, train_vectors, train["target"], cv=3, scoring="f1")
print(scores)
scores = model_selection.cross_val_score(rnd_clf, train_vectors, train["target"], cv=3, scoring="f1")
print(scores)
scores = model_selection.cross_val_score(voting_clf, train_vectors, train["target"], cv=3, scoring="f1")
print(scores)

[0.74925075 0.73884712 0.74569402]
[0.7445932  0.73582013 0.73137461]
[0.7106267  0.69839513 0.70749043]
[0.74769231 0.73857216 0.7457276 ]


In [32]:
# Evaluate (confusion metrix)
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

voting_clf.fit(train_vectors, train["target"])
y_train_pred = voting_clf.predict(train_vectors)
print(confusion_matrix(train["target"],y_train_pred,labels=[0,1]))
print(precision_score(train["target"],y_train_pred))
print(recall_score(train["target"],y_train_pred))
f1_score(train["target"],y_train_pred)

[[4309   33]
 [ 194 3077]]
0.9893890675241157
0.9406909202078875


0.9644256386146371

In [34]:
sample_submission = pd.read_csv("tweetsDisaster/sample_submission.csv")

In [36]:
sample_submission["target"] = voting_clf.predict(test_vectors)

In [37]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [38]:
sample_submission.to_csv("submission.csv", index=False)